# Adaptive PDE discretizations on Cartesian grids
## Volume : Reproducible research
## Part : Eikonal CPU/GPU solvers comparison
## Chapter : Voronoi's decomposition benchmark

This file uses the agd library to decompose (a large number of) symmetric positive definite matrices in dimension up to 6, drawn randomly. It illustrates the fact that this decomposition is fast enough for e.g. applications in PDE discretizations, where such a matrix has to be decomposed at each discretization point.

Please have a look at the following notebooks for more information on Voronoi's decomposition : [dimension 2 and 3](https://nbviewer.org/github/Mirebeau/AdaptiveGridDiscretizations_showcase/blob/master/Notebooks_Algo/TensorSelling.ipynb), [dimension 4 and 5](https://nbviewer.org/github/Mirebeau/AdaptiveGridDiscretizations_showcase/blob/master/Notebooks_Algo/TensorVoronoi.ipynb), 
[dimension 6](https://nbviewer.org/github/Mirebeau/AdaptiveGridDiscretizations_showcase/blob/master/Notebooks_Algo/TensorVoronoi6.ipynb).

[**Summary**](Summary.ipynb) of volume Reproducible research, this series of notebooks.

[**Main summary**](../Summary.ipynb) of the Adaptive Grid Discretizations 
	book of notebooks, including the other volumes.

# Table of contents
  * [1. Voronoi's decomposition](#1.-Voronoi's-decomposition)
    * [1.1 Setting up the test](#1.1-Setting-up-the-test)
    * [1.2 Benchmarking](#1.2-Benchmarking)



**Acknowledgement.** Some of the experiments presented in these notebooks are part of 
ongoing research with Ludovic Métivier and Da Chen.

Copyright Jean-Marie Mirebeau, Centre Borelli, ENS Paris-Saclay, CNRS, University Paris-Saclay

## 0. Importing the required libraries

In [1]:
import sys; sys.path.insert(0,"..")
#from Miscellaneous import TocTools; print(TocTools.displayTOC('Voronoi_Benchmark','Repro'))

In [2]:
from agd import LinearParallel as lp
from agd import AutomaticDifferentiation as ad
from agd.Eikonal import VoronoiDecomposition

In [3]:
import numpy as np; allclose = np.allclose
import time

### 0.1 Additional configuration

Uncomment this line to run on the GPU.

In [4]:
#VoronoiDecomposition.default_mode = 'gpu_transfer'

Choose the datasize for the benchmark.

In [5]:
nbench = 50000 if VoronoiDecomposition.default_mode == 'gpu_transfer' else 5000
#nbench = 500000 # Larger instance used in the paper

## 1. Voronoi's decomposition

### 1.1 Setting up the test

In [6]:
def MakeRandomTensor(dim, shape=tuple(), relax=0.05):
    A = np.random.standard_normal( (dim,dim) + shape )
    D = lp.dot_AA(lp.transpose(A),A)
    identity = np.eye(dim).reshape((dim,dim)+(1,)*len(shape))
    return D+lp.trace(D)*relax*identity

In [7]:
def Reconstruct(coefs,offsets):
     return lp.mult(coefs,lp.outer_self(offsets)).sum(2)
def LInfNorm(a):
    return np.max(np.abs(a))

In [8]:
np.random.seed(42) # Reproducibility
D = MakeRandomTensor(6)
np.linalg.eigvals(D)

array([13.76854937, 10.3988981 ,  7.96726883,  1.60586544,  4.28139694,
        3.09867544])

In [9]:
coefs,offsets = VoronoiDecomposition(D,retry64_tol=0.)

In [10]:
print("Coefficients : ", coefs)
print("Offsets : \n", offsets.astype(int))

Coefficients :  [0.06272357 0.08321161 0.09108027 0.17300802 0.25543099 0.33765603
 0.37974797 0.43206798 0.57262901 0.60285459 0.90199881 1.01424559
 1.02162947 1.14710001 1.20004529 1.32672471 1.71589609 1.77768024
 2.17099232 2.50835894 6.57177511]
Offsets : 
 [[ 1  1  1  1  0  1  1  1  0  1  1  0  1  0  0  0  0  1  0  0  0]
 [ 1  1  0  0  1  0 -1  0  1  0  0  1  1  0  0  1  0  0  0  0  1]
 [ 0  0 -1 -1  1 -1 -1 -1  0  0  1  1 -1  1  1  0  1  0  0  0  0]
 [ 1  0  0  0  0  1  1  1  0  0  1  0  0  0  1  0  0  0  0  1  0]
 [ 0  0  0 -1  1 -1 -1 -1  0  0  0  1  0  0  0  1  0  0  0  0  0]
 [ 0  0  1  0 -1  1  1  0 -1  1  0  0  0 -1  0  0  0  0  1  0  0]]


In [11]:
print("Minimal coefficient : ", np.min(coefs))
print("Reconstruction error : ", LInfNorm(D-Reconstruct(coefs,offsets)))
assert np.allclose(D,Reconstruct(coefs,offsets))

Minimal coefficient :  0.06272357357326566
Reconstruction error :  5.329070518200751e-15


### 1.2 Benchmarking

In [12]:
def decomp_time(n,dim,relax=0.01,atol=1e-4):
    np.random.seed(42) #Reproducibility
    D = MakeRandomTensor(dim,(n,),relax)
    start = time.time()
    coefs,offsets = VoronoiDecomposition(D)
#    print(coefs.reshape(-1)[0])
    print(f"Decomposition of {n} matrices completed in {time.time()-start} seconds")
    error = np.abs(D-Reconstruct(coefs,offsets))
    print("Tensor shape: ",D.shape,", max reconstruction error : ",np.max(error))
    assert np.allclose(D,Reconstruct(coefs,offsets),atol=atol)
    return D,coefs,offsets,error

In [13]:
for i in range(3): decomp_time(nbench,6,atol=1e-3)

Decomposition of 5000 matrices completed in 3.789393186569214 seconds
Tensor shape:  (6, 6, 5000) , max reconstruction error :  4.298783551348606e-13


Decomposition of 5000 matrices completed in 3.781599998474121 seconds
Tensor shape:  (6, 6, 5000) , max reconstruction error :  4.298783551348606e-13


Decomposition of 5000 matrices completed in 3.779866933822632 seconds
Tensor shape:  (6, 6, 5000) , max reconstruction error :  4.298783551348606e-13


In [14]:
for i in range(3): decomp_time(nbench,5);

Decomposition of 5000 matrices completed in 0.0732889175415039 seconds
Tensor shape:  (5, 5, 5000) , max reconstruction error :  4.440892098500626e-14


Decomposition of 5000 matrices completed in 0.07373213768005371 seconds
Tensor shape:  (5, 5, 5000) , max reconstruction error :  4.440892098500626e-14
Decomposition of 5000 matrices completed in 0.07318806648254395 seconds


Tensor shape:  (5, 5, 5000) , max reconstruction error :  4.440892098500626e-14


In [15]:
for i in range(3): decomp_time(nbench,4);

Decomposition of 5000 matrices completed in 0.020168304443359375 seconds
Tensor shape:  (4, 4, 5000) , max reconstruction error :  3.019806626980426e-14
Decomposition of 5000 matrices completed in 0.019492149353027344 seconds
Tensor shape:  (4, 4, 5000) , max reconstruction error :  3.019806626980426e-14


Decomposition of 5000 matrices completed in 0.01927018165588379 seconds
Tensor shape:  (4, 4, 5000) , max reconstruction error :  3.019806626980426e-14


In [16]:
for i in range(3): decomp_time(nbench,3);

Decomposition of 5000 matrices completed in 0.010003805160522461 seconds
Tensor shape:  (3, 3, 5000) , max reconstruction error :  1.7763568394002505e-14
Decomposition of 5000 matrices completed in 0.009601116180419922 seconds
Tensor shape:  (3, 3, 5000) , max reconstruction error :  1.7763568394002505e-14
Decomposition of 5000 matrices completed in 0.009795665740966797 seconds
Tensor shape:  (3, 3, 5000) , max reconstruction error :  1.7763568394002505e-14


In [17]:
for i in range(3): decomp_time(nbench,2);

Decomposition of 5000 matrices completed in 0.0066070556640625 seconds
Tensor shape:  (2, 2, 5000) , max reconstruction error :  3.552713678800501e-15
Decomposition of 5000 matrices completed in 0.0058939456939697266 seconds
Tensor shape:  (2, 2, 5000) , max reconstruction error :  3.552713678800501e-15
Decomposition of 5000 matrices completed in 0.00586700439453125 seconds
Tensor shape:  (2, 2, 5000) , max reconstruction error :  3.552713678800501e-15
